In [3]:
from binance.client import Client

In [1]:
from services.Binance import Binance

In [5]:
api = '5fSLEZVY3r6xBaWgreYOZXtfH6FUIh4fPGbjMIgWYB4e8IkZD9GOj4WZpi7EKxDH'
secret = 'vh7R58GkP80vMOaeyiHcpsiNtGGqBUR13XDP4cuwaLtXq8rsEq2mewJYbfn3PKm1'

In [6]:
client = Client(api, secret)

In [7]:
client.get_withdraw_history()

BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.